# Zemel et al. pre-processing fairness intervention

Zemel et al. (2013) proposes a clustering method which transforms the original data set by expressing points as linear combinations of learnt cluster centres. The transformed data set is as close as possible to the original while containing as little information as possible about the sensitive attributes. Thereby, demographic parity is achieved.

The output of their method includes besides a fair data representation also fair label predictions, which allows the comparison according to the usual fairness metrics. We apply their approach as implemented by IBM's AIF360 fairness tool box.

In [ ]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.datasets import StandardDataset
from helpers.fairness_measures import *
from helpers.finance import preprocess
from helpers.plot import group_box_plots, group_roc_curves

In [ ]:
from helpers import export_plot

## Load data

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "adult"
preprocess(data_dir)

In [ ]:
train = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

### Remove points which are not white/black people

In [ ]:
for data in [train, test, val]:
    data = data[data.race_white + data.race_black == 1]
    data.drop(
        [
            "race_amer_indian_eskimo",
            "race_asian_pac_islander",
            "race_other",
            "race_black",
        ],
        axis=1,
    )

## Set up fairness intervention
AIF360 requires expressing the original data sets via the "StandardDataset" class.

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("race_white")

In [ ]:
privileged_groups = [{"race_white": 1.0}]
unprivileged_groups = [{"race_white": 0.0}]

In [ ]:
val.head()

## Learn fair representation
The hyperparameters $A_x, A_y, A_z$ and $k$ are chosen how? According to optimum of a grid search for Adult.

In [ ]:
TR = joblib.load(artifacts_dir / "models" / "finance" / "zemel-race.pkl")

In [ ]:
# TR = LFR(
#     unprivileged_groups=unprivileged_groups,
#     privileged_groups=privileged_groups,
#     k=5,
#     Ax=0.01,
#     Ay=1.0,
#     Az=100.0,
# )
# TR = TR.fit(train_sds)

### Apply transformation to validation data

In [ ]:
transf_val_sds = TR.transform(val_sds)

## Evaluate fairness and accuracy

In [ ]:
# Fair labels
val_fair_labels = transf_val_sds.labels.flatten()

In [ ]:
print("Accuracy =", accuracy(val_fair_labels, val.salary))
print(
    "Black accuracy =",
    accuracy(
        val_fair_labels[val.race_white == 0], val.salary[val.race_white == 0],
    ),
)
print(
    "White accuracy =",
    accuracy(
        val_fair_labels[val.race_white == 1], val.salary[val.race_white == 1],
    ),
)
print(
    "Mean black score =", val_fair_labels[val.race_white == 0].mean(),
)
print(
    "Mean white score =", val_fair_labels[val.race_white == 1].mean(),
)

### Demographic parity

In [ ]:
di_d = disparate_impact_d(val_fair_labels, val.race_white)
print("Race demographic parity =", di_d)

In [ ]:
dp_bar = go.Figure(
    data=[
        go.Bar(
            x=[race],
            y=[val_fair_labels[val.race_white == race].mean()],
            name="White" if race else "Black",
        )
        for race in range(2)
    ],
    layout={"yaxis": {"range": [0, 1]}},
)
dp_bar

In [ ]:
export_plot(dp_bar, "zemel-race-dp.json")